In [2]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np
import scipy.io
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
# load data
data = scipy.io.loadmat('game_data.mat')
x = np.concatenate((data['a'],data['b']),axis=1)

In [5]:
data = pd.DataFrame(x,columns=['buy','sell'])
data.index = pd.to_datetime(data.index,unit='s')
data.head()

,buy,sell
1970-01-01 00:00:00,34.2,16.0
1970-01-01 00:00:01,34.2,16.0
1970-01-01 00:00:02,2.0,-2.8
1970-01-01 00:00:03,4.6,2.6
1970-01-01 00:00:04,0.6,-8.0


In [6]:
data.profit = data.sell - data.buy - 0.2

In [100]:
changes = np.diff(data.profit) > 0

X = deque(maxlen=1000) # Independent, or input variables
Y = deque(maxlen=1000)
type(changes)
# X = deque(maxlen=500) # Independent, or input variables
# Y = deque(maxlen=500)

numpy.ndarray

In [101]:
for i in range (200,len(changes)):
    X.append(changes[i-10:i-1]) # Add independent variables, the prior changes
    Y.append(changes[i]) 
#     if len(X) = 500:
#         classifier.fit(np.asarray(X_train),np.asarray(Y_train)) # Generate the model
#         # Add dependent variable, the final change
    
type(X)
type(Y)

collections.deque

In [102]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)


In [103]:
classifier = RandomForestClassifier(max_depth=3,n_estimators=10)

In [104]:
classifier.fit(np.asarray(X_train),np.asarray(Y_train)) # Generate the model


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [105]:
predict = []
prediction = classifier.predict(X_test) # Predict
predict.append(prediction)

In [106]:
float(sum(np.abs(prediction.astype(int) - np.array(Y_test).astype(int))))/len(prediction)

0.462

In [107]:
float(sum(np.array(Y_test).astype(int)))/len(prediction)

0.408

In [99]:
len(changes)

324029

In [38]:
np.array(X)

array([array([0]), array([0, 1]), array([0, 1, 1]), array([0, 1, 1, 0]),
       array([0, 1, 1, 0, 1]), array([0, 1, 1, 0, 1, 1]),
       array([0, 1, 1, 0, 1, 1, 0]), array([0, 1, 1, 0, 1, 1, 0, 1]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]),
       array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1]),
      

In [ ]:
# Use the previous 10 bars' movements to predict the next movement.

# Use a random forest classifier. More here: http://scikit-learn.org/stable/user_guide.html


def initialize(context):
    context.security = sid(698) # Boeing
    context.window_length = 10 # Amount of prior bars to study
    
    context.classifier = RandomForestClassifier() # Use a random forest classifier
    
    # deques are lists with a maximum length where old entries are shifted out
    context.recent_prices = deque(maxlen=context.window_length+2) # Stores recent prices
    context.X = deque(maxlen=500) # Independent, or input variables
    context.Y = deque(maxlen=500) # Dependent, or output variable
    
    context.prediction = 0 # Stores most recent prediction
    
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_close(minutes=5))
    schedule_function(record_vars, date_rules.every_day(), time_rules.market_close())
    
def rebalance(context, data):
    context.recent_prices.append(data.current(context.security, 'price')) # Update the recent prices
    if len(context.recent_prices) == context.window_length+2: # If there's enough recent price data
        
        # Make a list of 1's and 0's, 1 when the price increased from the prior bar
        changes = np.diff(context.recent_prices) > 0
        
        context.X.append(changes[:-1]) # Add independent variables, the prior changes
        context.Y.append(changes[-1]) # Add dependent variable, the final change
        
        if len(context.Y) >= 100: # There needs to be enough data points to make a good model
            
            context.classifier.fit(context.X, context.Y) # Generate the model
            
            context.prediction = context.classifier.predict(changes[1:]) # Predict
            
            # If prediction = 1, buy all shares affordable, if 0 sell all shares
            order_target_percent(context.security, context.prediction)
                
def record_vars(context, data):
    record(prediction=int(context.prediction))  
